In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": 0.1}], "evaluations": [{"state": 0, "time": 2}]}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.5654729008674622],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [0.04317250847816467],
 'stddev': [2.95021390914917],
 'time_points': [2.0],
 'time_steps': [9]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.30198344588279724,
  0.5003674626350403,
  1.104605793952942,
  2.1038992404937744],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [0.005186221096664667,
  0.0016712475335225463,
  0.00013882650819141418,
  0.0017001235391944647],
 'stddev': [2.0011544227600098,
  2.8310344219207764,
  4.47882604598999,
  6.332578182220459],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [99, 199, 499, 999]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.IndependentGaussian())
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.2852932810783386,
  0.3169647455215454,
  0.4717574715614319,
  0.4826488196849823,
  1.0938289165496826,
  1.118740200996399],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [0.014204268343746662,
  0.006675051525235176,
  -0.005391399841755629,
  0.003731429809704423,
  0.02145765721797943,
  -0.024841122329235077],
 'stddev': [1.9928534030914307,
  1.9958460330963135,
  2.8063087463378906,
  2.824434280395508,
  4.444184303283691,
  4.430912971496582],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [1999, 3999, 9999]}

In [9]:
results = sdk.run (model, server)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,1999,10000,0.303593,0.020038,2.003781,-0.036830
1,BrownianMotion,,1,1.0,1999,10000,0.266535,0.019836,1.983624,0.019266
2,BrownianMotion,,0,2.0,3999,10000,0.528057,0.028036,2.803569,-0.021210
3,BrownianMotion,,1,2.0,3999,10000,0.477577,0.027794,2.779386,-0.034546
4,BrownianMotion,,0,5.0,9999,10000,1.119712,0.044998,4.499833,-0.030536
5,BrownianMotion,,1,5.0,9999,10000,1.108240,0.044593,4.459272,-0.003268
